In [2]:
import numpy as np # 행렬 계산 라이브러리
import matplotlib.pyplot as pit # 그래프 보여주는 라이브러리
import scipy # 행렬 계산 라이브러리
import tensorflow as tf   # 텐서플로우 라이브러리
print "tensorflow load complete"

tensorflow load complete


In [3]:
from tensorflow.examples.tutorials.mnist import input_data  # 텐서플로우에서 기본제공하는 MNIST(숫자필기체, 1만개) 데이터입니다
mnist = input_data.read_data_sets('data/', one_hot=True) # OneHot Coding 이란, 정답 라벨 하나당 비트 하나를 할당하는것을 말합니다
trainimg = mnist.train.images # 학습할 이미지
trainlabel = mnist.train.labels # 학습할 라벨
testimg = mnist.test.images # 테스트할 이미지
testlabel = mnist.test.labels # 테스트할 라벨

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate = 0.01 # W를 Optimize 1회 할때, 가중치 비율 
training_epochs = 50  # 전체 학습 데이터에 대해서 총 반복학습 횟수
batch_size = 100  # 10,000개 데이터 중에 한번에 몇개를 가져와서 Optimize 할것인가
display_step = 1 # 몇번째 Epoch 마다 성능을 리포트 할것인가 

# 로지스틱 회귀의 그래프를 만들어 봅시다

### 텐서플로우의 구조는 추상적으로 아래와 같습니다

- | 먹이 : x |  -> | 레이어 | -> | 레이어 | -> ... -> | 레이어 | -> | 먹이 : y |  -> | COST | -> | 최적화 방법 |

그러니까, 레이어 그래프를 정의하고, COST를 정의하고 , 최적화 방법을 정의하고,

먹이 x 와 먹이 y 를 먹여(?)서, 그래프를 최적화 방법에 따라, COST를 최소화 하는 방향으로 레이어를 학습합니다

In [6]:
x = tf.placeholder("float", [None, 784])  # 먹이는 Placeholder 입니다. 먹이 x 입니다 (이미지 28x28 를 한줄로 핀 784 데이터) 
y = tf.placeholder("float", [None, 10]) # 먹이 y 입니다 (라벨)

W = tf.Variable(tf.zeros([784, 10])) # 784개의 데이터를 10개의 데이터로 연결하는 첫번쨰 layer 입니다
b = tf.Variable(tf.zeros([10])) # 첫번째 layer의 biases (그래프의 y 절편)

actv = tf.nn.softmax(tf.matmul(x, W) + b)  #  softmax ( x * W  + b )  ->  softmax 는 결과값을 0에서 1사이로 스케일을 바꿔주는 알고리즘 입니다 (모두합하면 1)
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices=1)) # - y * log(y') = 크로스엔트로피 ( 낮을수록 비슷한 것 ) 

optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # GradientDescentOptimizer (최적화 전략 중 하나) 는 cost를 최소화 하는방향으로
pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1)) # argmax 는 가장 높은 값의 index를 가져옴, 즉 (0.0 , 0.0, 0.9 , 0.0 , 0.1 ) 이면 argmax 값은 2 인덱스
accr = tf.reduce_mean(tf.cast(pred, "float")) # 맞췄는지 결과값은 batch 사이즈 만큼이므로 평균을 낸다. 이것은 정확도를 말한다

init = tf.initialize_all_variables() #그래프의 모든 Variable을 초기값을 준다 (여기서는 0에 가까운 랜덤으로 초기화됨)
print "network constructed"

network constructed


In [ ]:
with tf.Session() as sess:
    sess.run(init) # 그래프 구조중, 초기화 하는 코드를 실행한다. (위에 구조에 init 을 선언하였음)
    
    for epoch in range(training_epochs):  # training_epoch만큼 학습을 반복한다
        avg_cost = 0.
        num_batch = int(mnist.train.num_examples/batch_size) 
        for i in range(num_batch): # batch 사이즈 만큼 데이터를 가져와서 학습시키는것을 반복한다
            if 0:
                batch_xs, batch_ys = mnist.train.next_batch(batch_size) # batch 사이즈 만큼 데이터를 가져온다
            else:
                randidx = np.random.randint(trainimg.shape[0], size = batch_size) # 데이터를 랜덤하게 섞어서 가져오기 위한 랜덤인덱스
                batch_xs = trainimg[randidx, :] # batch 사이즈 만큼의 랜덤 인덱스들의 x 데이터만 뽑아서 가져온다
                batch_ys = trainlabel[randidx, :] # batch 사이즈 만큼의 랜덤 인덱스들의 y 데이터만 뽑아서 가져온다
                
            sess.run(optm, feed_dict={x:batch_xs, y:batch_ys}) # optimize 를 실행한다. 이때 가져온 x 와 y를 먹이로 줘야된다
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys})/num_batch #  현재 Cost를 실행한다. 이때 가져온 x 와 y를 먹이로 줘야된다 
            
        if epoch % display_step == 0:
            train_acc = accr.eval({x:batch_xs, y:batch_ys}) # 현재 정확도를 구하도록 실행한다. 이때 가져온 x 와 y를 먹이로 줘야된다
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f"
                  % (epoch, training_epochs, avg_cost, train_acc)) # 정확도를 출력한다.
    print "Optimization Finished!"
    print "Accuracy : ", accr.eval({x:mnist.test.images, y:mnist.test.labels}) # 학습이 끝나면 최종적으로 테스트 데이터를 먹이로 줘서 정확도를 구하도록한다
print "Done. " # 끝.

Epoch: 000/050 cost: 1.171730226 train_acc: 0.820
Epoch: 001/050 cost: 0.655319740 train_acc: 0.860
Epoch: 002/050 cost: 0.549059332 train_acc: 0.830
Epoch: 003/050 cost: 0.491943460 train_acc: 0.900
Epoch: 004/050 cost: 0.461978950 train_acc: 0.920
Epoch: 005/050 cost: 0.437448837 train_acc: 0.870
Epoch: 006/050 cost: 0.423181105 train_acc: 0.900
Epoch: 007/050 cost: 0.408722305 train_acc: 0.920
Epoch: 008/050 cost: 0.399912114 train_acc: 0.910
Epoch: 009/050 cost: 0.393934822 train_acc: 0.870
Epoch: 010/050 cost: 0.387194715 train_acc: 0.890
Epoch: 011/050 cost: 0.376580075 train_acc: 0.920
Epoch: 012/050 cost: 0.374908861 train_acc: 0.910
Epoch: 013/050 cost: 0.368051432 train_acc: 0.890
Epoch: 014/050 cost: 0.362041670 train_acc: 0.930
Epoch: 015/050 cost: 0.356530867 train_acc: 0.930
Epoch: 016/050 cost: 0.354171366 train_acc: 0.880
Epoch: 017/050 cost: 0.351668806 train_acc: 0.920
Epoch: 018/050 cost: 0.347968256 train_acc: 0.930
Epoch: 019/050 cost: 0.347735364 train_acc: 0.890
